In [33]:
%%capture pip_output
!pip3 install -r requirements.txt

%load_ext dotenv
%dotenv

In [35]:
# Confirm that the install worked and we have an API key
if pip_output.stderr:
    print("An error occurred during PIP install or dotenv load:\n", pip_output.stderr)
else:
    print("PIP Install successful, no errors reported.")

import os
api_key = os.getenv('OPENAI_API_KEY')
print("API Key Loaded:", api_key is not None)
print("API key starts with:", api_key[:5])

PIP Install successful, no errors reported.
API Key Loaded: True
API key starts with: sk-7q


In [37]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

logging.getLogger().handlers.clear()
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [24]:
# Download the data files
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'


--2023-12-19 10:48:02--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: 'data/paul_graham/paul_graham_essay.txt'

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2023-12-19 10:48:03 (4.63 MB/s) - 'data/paul_graham/paul_graham_essay.txt' saved [75042/75042]



In [40]:
reader = SimpleDirectoryReader("./data/paul_graham/")
documents = reader.load_data()

Load the content from ./data into the index, storing that index locally locally to avoid rebuilding it each time it is needed. 

Delete the ./storage directory to force index rebuild (e.g. when new data is added).

In [47]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
if not os.path.exists("./storage"):
    # load the documents and create the index
    print("Creating index")
    documents = SimpleDirectoryReader("data/paul_graham").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    print("Storing index")
    index.storage_context.persist()
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)
    print("Loaded index from storage")



Loading all indices.
Loaded index from storage


In [38]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The author, growing up, worked on writing and programming. They wrote short stories and tried writing programs on an IBM 1401 computer. They later got a microcomputer and started programming more extensively, writing simple games and a word processor.
